In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline

from astroML.plotting          import hist
from astropy.io                import fits
from astropy.modeling          import models, fitting
from datetime                  import datetime
from image_registration        import cross_correlation_shifts
from glob                      import glob
from matplotlib.ticker         import MaxNLocator
from matplotlib                import style
from os                        import listdir
from pandas                    import DataFrame, read_csv, read_pickle, scatter_matrix
from photutils                 import CircularAperture, CircularAnnulus, aperture_photometry, findstars
from least_asymmetry           import actr, moments, fitgaussian
from pylab                     import ion, gcf, sort, linspace, indices, median, mean, std, empty, figure, transpose, ceil
from pylab                     import concatenate, pi, sqrt, ones, diag, inf, rcParams, isnan, isfinite, array, nanmax
from numpy                     import min as npmin, max as npmax, zeros, arange, sum, float, isnan, hstack
from numpy                     import int32 as npint, round as npround, nansum as sum, nanstd as std
from os                        import environ
from seaborn                   import *
from scipy.special             import erf
from scipy                     import stats
from sklearn.externals         import joblib
from socket                    import gethostname
from statsmodels.robust        import scale
from statsmodels.nonparametric import kde
from sys                       import exit
from time                      import time, localtime

from numpy                     import zeros, nanmedian as median, nanmean as mean, nan
from sys                       import exit
from sklearn.externals         import joblib
from least_asymmetry           import actr

import numpy as np

**Master Class for Exoplanet Time Series Observation Photometry**

In [3]:
from ExoplanetTSO_Auxiliary import wanderer

In [4]:
rcParams['image.interpolation'] = 'None'
rcParams['image.cmap']          = 'Blues_r'
rcParams['axes.grid']           = False

In [5]:
ls /Users/jonathan/Research/Planets/HATP26/data/raw/r50677248/ch2/bcd

SPITZER_I2_50677248_0000_0000_3_bcd.fits
SPITZER_I2_50677248_0000_0000_3_bcd.log
SPITZER_I2_50677248_0000_0000_3_bimsk.fits
SPITZER_I2_50677248_0000_0000_3_bunc.fits
SPITZER_I2_50677248_0000_0000_3_cov2d.fits
SPITZER_I2_50677248_0000_0000_3_msk2d.fits
SPITZER_I2_50677248_0000_0000_3_ptn.log
SPITZER_I2_50677248_0000_0000_3_sub2d.fits
SPITZER_I2_50677248_0000_0000_3_unc2d.fits
SPITZER_I2_50677248_0001_0000_3_bcd.fits
SPITZER_I2_50677248_0001_0000_3_bcd.log
SPITZER_I2_50677248_0001_0000_3_bimsk.fits
SPITZER_I2_50677248_0001_0000_3_bunc.fits
SPITZER_I2_50677248_0001_0000_3_cov2d.fits
SPITZER_I2_50677248_0001_0000_3_msk2d.fits
SPITZER_I2_50677248_0001_0000_3_ptn.log
SPITZER_I2_50677248_0001_0000_3_sub2d.fits
SPITZER_I2_50677248_0001_0000_3_unc2d.fits
SPITZER_I2_50677248_0002_0000_3_bcd.fits
SPITZER_I2_50677248_0002_0000_3_bcd.log
SPITZER_I2_50677248_0002_0000_3_bimsk.fits
SPITZER_I2_50677248_0002_0000_3_bunc.fits
SPITZER_I2_50677248_0002_0000_3_cov2d.fits
SPITZER_I2_50677248_0002_0000_3_msk

SPITZER_I2_50677248_0066_0000_3_bcd.log
SPITZER_I2_50677248_0066_0000_3_bimsk.fits
SPITZER_I2_50677248_0066_0000_3_bunc.fits
SPITZER_I2_50677248_0066_0000_3_cov2d.fits
SPITZER_I2_50677248_0066_0000_3_msk2d.fits
SPITZER_I2_50677248_0066_0000_3_ptn.log
SPITZER_I2_50677248_0066_0000_3_sub2d.fits
SPITZER_I2_50677248_0066_0000_3_unc2d.fits
SPITZER_I2_50677248_0067_0000_3_bcd.fits
SPITZER_I2_50677248_0067_0000_3_bcd.log
SPITZER_I2_50677248_0067_0000_3_bimsk.fits
SPITZER_I2_50677248_0067_0000_3_bunc.fits
SPITZER_I2_50677248_0067_0000_3_cov2d.fits
SPITZER_I2_50677248_0067_0000_3_msk2d.fits
SPITZER_I2_50677248_0067_0000_3_ptn.log
SPITZER_I2_50677248_0067_0000_3_sub2d.fits
SPITZER_I2_50677248_0067_0000_3_unc2d.fits
SPITZER_I2_50677248_0068_0000_3_bcd.fits
SPITZER_I2_50677248_0068_0000_3_bcd.log
SPITZER_I2_50677248_0068_0000_3_bimsk.fits
SPITZER_I2_50677248_0068_0000_3_bunc.fits
SPITZER_I2_50677248_0068_0000_3_cov2d.fits
SPITZER_I2_50677248_0068_0000_3_msk2d.fits
SPITZER_I2_50677248_0068_0000_3_p

As an example, Spitzer data is expected to be store in the directory structure:

`$HOME/PLANET_DIRECTORY/PLANETNAME/data/raw/AORDIR/CHANNEL/bcd/`

EXAMPLE:

1. On a Linux machine
2. With user `tempuser`,
3. And all Spitzer data is store in `Research/Planets`
4. The planet named `Happy-5b`
5. Observed during AOR r11235813
6. In CH2 (4.5 microns)

The `loadfitsdir` should read as: `/home/tempuser/Research/Planets/HAPPY5/data/raw/r11235813/ch2/bcd/`

In [6]:
planetName      = 'HATP26'
planetDirectory = '/Research/Planets/'

dataDir     = environ['HOME'] + planetDirectory + planetName + '/data/raw/'

# This is only a list as an example because it the following code can be used to combine multiple AORs
AORs    = ['r11235813'] 

AORs    = ['r50677248/', 'r50676992/', 'r50676736/', 'r50676480/', \
           'r50667264/', 'r50667008/', 'r42624768/', 'r42621184/']


channel = 'ch2/'

dataSub = 'bcd/'

fileExt = '*bcd.fits'
uncsExt = '*bunc.fits'

In [7]:
loadfitsdir = dataDir + AORs[0] + '/' + channel + dataSub
print(loadfitsdir)

/Users/jonathan/Research/Planets/HATP26/data/raw/r50677248//ch2/bcd/


In [8]:
fitsFilenames = glob(loadfitsdir + fileExt)
uncsFilenames = glob(loadfitsdir + uncsExt)

In [9]:
fitsFilenames

['/Users/jonathan/Research/Planets/HATP26/data/raw/r50677248//ch2/bcd/SPITZER_I2_50677248_0000_0000_3_bcd.fits',
 '/Users/jonathan/Research/Planets/HATP26/data/raw/r50677248//ch2/bcd/SPITZER_I2_50677248_0001_0000_3_bcd.fits',
 '/Users/jonathan/Research/Planets/HATP26/data/raw/r50677248//ch2/bcd/SPITZER_I2_50677248_0002_0000_3_bcd.fits',
 '/Users/jonathan/Research/Planets/HATP26/data/raw/r50677248//ch2/bcd/SPITZER_I2_50677248_0003_0000_3_bcd.fits',
 '/Users/jonathan/Research/Planets/HATP26/data/raw/r50677248//ch2/bcd/SPITZER_I2_50677248_0004_0000_3_bcd.fits',
 '/Users/jonathan/Research/Planets/HATP26/data/raw/r50677248//ch2/bcd/SPITZER_I2_50677248_0005_0000_3_bcd.fits',
 '/Users/jonathan/Research/Planets/HATP26/data/raw/r50677248//ch2/bcd/SPITZER_I2_50677248_0006_0000_3_bcd.fits',
 '/Users/jonathan/Research/Planets/HATP26/data/raw/r50677248//ch2/bcd/SPITZER_I2_50677248_0007_0000_3_bcd.fits',
 '/Users/jonathan/Research/Planets/HATP26/data/raw/r50677248//ch2/bcd/SPITZER_I2_50677248_0008_0

In [10]:
uncsFilenames

['/Users/jonathan/Research/Planets/HATP26/data/raw/r50677248//ch2/bcd/SPITZER_I2_50677248_0000_0000_3_bunc.fits',
 '/Users/jonathan/Research/Planets/HATP26/data/raw/r50677248//ch2/bcd/SPITZER_I2_50677248_0001_0000_3_bunc.fits',
 '/Users/jonathan/Research/Planets/HATP26/data/raw/r50677248//ch2/bcd/SPITZER_I2_50677248_0002_0000_3_bunc.fits',
 '/Users/jonathan/Research/Planets/HATP26/data/raw/r50677248//ch2/bcd/SPITZER_I2_50677248_0003_0000_3_bunc.fits',
 '/Users/jonathan/Research/Planets/HATP26/data/raw/r50677248//ch2/bcd/SPITZER_I2_50677248_0004_0000_3_bunc.fits',
 '/Users/jonathan/Research/Planets/HATP26/data/raw/r50677248//ch2/bcd/SPITZER_I2_50677248_0005_0000_3_bunc.fits',
 '/Users/jonathan/Research/Planets/HATP26/data/raw/r50677248//ch2/bcd/SPITZER_I2_50677248_0006_0000_3_bunc.fits',
 '/Users/jonathan/Research/Planets/HATP26/data/raw/r50677248//ch2/bcd/SPITZER_I2_50677248_0007_0000_3_bunc.fits',
 '/Users/jonathan/Research/Planets/HATP26/data/raw/r50677248//ch2/bcd/SPITZER_I2_5067724

# Load ExoplanetTSO Class

Necessary Constants Spitzer
---

In [11]:
ppm             = 1e6
y,x             = 0,1

yguess, xguess  = 15., 15.   # Specific to Spitzer circa 2010 and beyond
filetype        = 'bcd.fits' # Specific to Spitzer Basic Calibrated Data

Load Stored Instance from Save Files
---

If this is **not** your time, then you can load a saved file in the `loadfitsdir` directory that has the filename:

`PLANETNAME_Wanderer_METHOD_DATE.pickle.save` 

```python
method = 'mean'
example_wanderer_mean = wanderer(loadfitsdir=loadfitsdir, filetype=filetype, 
                                            yguess=yguess, xguess=xguess, method=method)

example_wanderer_mean.load_data_from_save_files(savefiledir='./SaveFiles/', 
                        saveFileNameHeader=planetName+'_Wanderer_Mean_', saveFileType='.pickle.save')
```

```python
method = 'median'
example_wanderer_median = wanderer(loadfitsdir=loadfitsdir_ModA, filetype=filetype, 
                                            yguess=yguess, xguess=xguess, method=method)

example_wanderer_median.load_data_from_save_files(savefiledir='./SaveFiles/', saveFileNameHeader='Example_Wanderer_Median_', saveFileType='.pickle.save')
```

Start a New Instance with Median for the Metric
---

In [12]:
method = 'median'

print('Initialize an instance of `wanderer` as `example_wanderer_median`\n')
example_wanderer_median = wanderer(fitsFileDir=loadfitsdir, filetype=filetype, telescope='Spitzer',
                                            yguess=yguess, xguess=xguess, method=method)

Initialize an instance of `wanderer` as `example_wanderer_median`



** Not all who wander are lost **


Completed Class Definition at 2017-10-13 0h55m29s


In [13]:
print('Load Data From Fits Files in ' + loadfitsdir + '\n')
example_wanderer_median.load_data_from_fits_files()

Load Data From Fits Files in /Users/jonathan/Research/Planets/HATP26/data/raw/r50677248//ch2/bcd/



A Jupyter Widget

/Users/jonathan/Research/github/ExoplanetTSO/ExoplanetTSO_Auxiliary.py:348: RuntimeWarning: invalid value encountered in sqrt
  self.noiseCube[kfile * nFramesPerFile + iframe] = sqrt(fitsNow[0].data[iframe])


In [14]:
print('Skipping Load Data From Save Files in ' + loadfitsdir + '\n')
# example_wanderer_median.load_data_from_save_files()

Skipping Load Data From Save Files in /Users/jonathan/Research/Planets/HATP26/data/raw/r50677248//ch2/bcd/



In [15]:
print('Find, flag, and NaN the "Bad Pixels" Outliers' + '\n')
example_wanderer_median.find_bad_pixels()

Find, flag, and NaN the "Bad Pixels" Outliers



/Users/jonathan/anaconda3/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:879: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanmedian1d, axis, a, overwrite_input)
/Users/jonathan/anaconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:4016: RuntimeWarning: Invalid value encountered in median for 32 results
  r = func(a, **kwargs)


There are 4123 'Hot' Pixels


/Users/jonathan/Research/github/ExoplanetTSO/ExoplanetTSO_Auxiliary.py:631: RuntimeWarning: invalid value encountered in greater
  self.imageBadPixMasks = abs(self.imageCube - self.imageCubeMedian) > nSig*self.imageCubeMAD


In [16]:
print('Fit for All Centers: Flux Weighted, Gaussian Fitting, Gaussian Moments, Least Asymmetry' + '\n')
example_wanderer_median.fit_gaussian_centering()
example_wanderer_median.fit_flux_weighted_centering()
# example_wanderer_median.fit_least_asymmetry_centering()
# example_wanderer_median.fit_all_centering() # calling this calls least_asymmetry, which does not work :(

Fit for All Centers: Flux Weighted, Gaussian Fitting, Gaussian Moments, Least Asymmetry



A Jupyter Widget

/Users/jonathan/anaconda3/lib/python3.6/site-packages/scipy/optimize/minpack.py:427: RuntimeWarning: Number of calls to function has reached maxfev = 1400.
  warnings.warn(errors[info][0], RuntimeWarning)


A Jupyter Widget

In [17]:
print('Measure Background Estimates with All Methods: Circle Masked, Annular Masked, KDE Mode, Median Masked' + '\n')
# example_wanderer_median.measure_background_circle_masked()
# example_wanderer_median.measure_background_annular_mask()
# example_wanderer_median.measure_background_KDE_Mode()
# example_wanderer_median.measure_background_median_masked()
example_wanderer_median.measure_all_background()

Measure Background Estimates with All Methods: Circle Masked, Annular Masked, KDE Mode, Median Masked

Measuring Background Using Circle Mask


A Jupyter Widget

Measuring Background Using Annular Mask


A Jupyter Widget

Measuring Background Using Median Mask


A Jupyter Widget

/Users/jonathan/anaconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:4016: RuntimeWarning: Invalid value encountered in median
  r = func(a, **kwargs)
/Users/jonathan/Research/github/ExoplanetTSO/ExoplanetTSO_Auxiliary.py:977: RuntimeWarning: invalid value encountered in less
  medianMask= abs(self.imageCube[kframe] - medFrame) < nSig*madFrame
/Users/jonathan/anaconda3/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:990: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


Measuring Background Using KDE Mode


A Jupyter Widget

/Users/jonathan/anaconda3/lib/python3.6/site-packages/statsmodels/nonparametric/kde.py:454: RuntimeWarning: invalid value encountered in greater
  X = X[np.logical_and(X>clip[0], X<clip[1])] # won't work for two columns.
/Users/jonathan/anaconda3/lib/python3.6/site-packages/statsmodels/nonparametric/kde.py:454: RuntimeWarning: invalid value encountered in less
  X = X[np.logical_and(X>clip[0], X<clip[1])] # won't work for two columns.


In [18]:
example_wanderer_mean.measure_effective_width()

NameError: name 'example_wanderer_mean' is not defined

In [ ]:
print('Iterating over Background Techniques, Centering Techniques, Aperture Radii' + '\n')
centering_choices  = ['Gaussian_Fit']#, 'Gaussian_Mom', 'FluxWeighted']#, 'LeastAsymmetry']
background_choices = ['AnnularMask']#example_wanderer_median.background_df.columns
staticRads         = [2.5 ]# aperRads = np.arange(1, 6,0.5)
varRads            = [None]# aperRads = np.arange(1, 6,0.5)

example_wanderer_median.compute_flux_over_time_over_aperRad(staticRads, varRads, centering_choices, \
                                                            background_choices, useTheForce=True)

In [ ]:
print('Saving `example_wanderer_median` to a set of pickles for various Image Cubes and the Storage Dictionary')
example_wanderer_median.save_data_to_save_files(savefiledir='./SaveFiles/', \
                                                saveFileNameHeader='Example_Wanderer_Median_', \
                                                saveFileType='.pickle.save')

Start a New Instance with Mean for the Metric
---

In [ ]:
method = 'mean'

In [ ]:
print('Initialize an instance of `wanderer` as `example_wanderer_mean`')
example_wanderer_mean = wanderer(fitsFileDir=loadfitsdir, filetype = filetype, telescope='Spitzer',
                                yguess=yguess, xguess=xguess, method=method)

In [ ]:
print('Load Data From Fits Files in ' + loadfitsdir)
example_wanderer_mean.load_data_from_fits_files()

In [ ]:
print('Skipping Load Data From Save Files in ' + loadfitsdir)
# example_wanderer_mean.load_data_from_save_files()

In [ ]:
print('Find, flag, and NaN the "Bad Pixels" Outliers')
example_wanderer_mean.find_bad_pixels()

In [ ]:
print('Fit for All Centers: Flux Weighted, Gaussian Fitting, Gaussian Moments, Least Asymmetry')
example_wanderer_mean.fit_gaussian_centering()
example_wanderer_mean.fit_flux_weighted_centering()
# example_wanderer_mean.fit_least_asymmetry_centering()
# example_wanderer_mean.fit_all_centering()

In [ ]:
print('Measure Background Estimates with All Methods: Circle Masked, Annular Masked, KDE Mode, Median Masked')
# example_wanderer_mean.measure_background_circle_masked()
# example_wanderer_mean.measure_background_annular_mask()
# example_wanderer_mean.measure_background_KDE_Mode()
# example_wanderer_mean.measure_background_median_masked()
example_wanderer_mean.measure_all_background()

In [ ]:
example_wanderer_mean.measure_effective_width()

In [ ]:
print('Iterating over Background Techniques, Centering Techniques, Aperture Radii' + '\n')
background_choices = ['AnnularMask']#example_wanderer_mean.background_df.columns
centering_choices  = ['Gaussian_Fit']#, 'Gaussian_Mom', 'FluxWeighted']#, 'LeastAsymmetry']
aperRads           = [2.5] # aperRads = np.arange(1, 6,0.5)

example_wanderer_mean.compute_flux_over_time_over_aperRad(staticRads, varRads, centering_choices, \
                                                            background_choices, useTheForce=True)

In [ ]:
print('Saving `example_wanderer_mean` to a set of pickles for various Image Cubes and the Storage Dictionary')
example_wanderer_mean.save_data_to_save_files(savefiledir='./SaveFiles/', saveFileNameHeader='Example_Wanderer_Mean_', saveFileType='.pickle.save')